In [13]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import gzip
from collections import defaultdict
import random

from sklearn.feature_extraction.text import TfidfVectorizer
import urllib
import scipy.optimize
from collections import defaultdict # Dictionaries with default values
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import ast

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /home/j1feng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
rent = pd.read_csv('clean.csv')
rent.review_summary = rent.review_summary.fillna('')
rent.review_text = rent.review_text.fillna('')

In [3]:
rent = rent.sample(frac=1).reset_index(drop=True)
train = rent[:180000]
valid = rent[180000:]

In [4]:
y_train = train.fit
y_valid = valid.fit

In [6]:
tfidf = TfidfVectorizer(stop_words = 'english', sublinear_tf = True, max_features = 1000, tokenizer = word_tokenize) 
Train_X_summary = tfidf.fit_transform(train.review_summary).toarray()

In [7]:
tfidf2 = TfidfVectorizer(stop_words = 'english', sublinear_tf = True, max_features = 1000, tokenizer = word_tokenize) 
Train_X_text = tfidf2.fit_transform(train.review_text).toarray()

In [8]:
combined_text = np.hstack((Train_X_summary, Train_X_text))

In [10]:
combined_text.shape

(180000, 2000)

In [14]:
clf = LogisticRegression()
clf.fit(combined_text, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
Valid_X_summary = tfidf.transform(valid.review_summary).toarray()

In [16]:
Valid_X_text = tfidf2.transform(valid.review_text).toarray()

In [17]:
combined_text_valid = np.hstack((Valid_X_summary, Valid_X_text))

In [18]:
clf.score(combined_text_valid, y_valid)

0.8058035714285714

In [20]:
y_valid.value_counts()

fit      9235
small    1698
large    1611
Name: fit, dtype: int64

In [21]:
9235 / (len(y_valid))

0.7362085459183674

In [22]:
clf = LogisticRegression(max_iter = 10000, C=0.1)
clf.fit(combined_text, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
clf.score(combined_text_valid, y_valid)

0.8038903061224489